In [1]:
import os
import random
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np
from matplotlib.ticker import MaxNLocator
from matplotlib.image import imread
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import EfficientNetB0

2025-07-17 01:41:29.386354: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752716489.585180      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752716489.648764      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
train_path = "/kaggle/input/fruit-classification-dataset/Fruit_dataset/train1"
val_path = "/kaggle/input/fruit-classification-dataset/Fruit_dataset/val1"

BATCH_SIZE = 32
IMG_SIZE = (224, 224)

#创建训练集
train_dataset = image_dataset_from_directory(
    train_path,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    label_mode='categorical',
    seed=42
)

#创建验证集
val_dataset = image_dataset_from_directory(
    val_path,
    shuffle=False,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    label_mode='categorical',
    seed=42
)

class_names = train_dataset.class_names

AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=AUTOTUNE)


print(f"Number of classes: {len(class_names)}")
print(f"Class names: {class_names}")
print(f"Training batches: {tf.data.experimental.cardinality(train_dataset)}")
print(f"Validation batches: {tf.data.experimental.cardinality(val_dataset)}")

Found 40000 files belonging to 100 classes.


I0000 00:00:1752716536.503575      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Found 5000 files belonging to 100 classes.
Number of classes: 100
Class names: ['abiu', 'acai', 'acerola', 'ackee', 'ambarella', 'apple', 'apricot', 'avocado', 'banana', 'barbadine', 'barberry', 'betel_nut', 'bitter_gourd', 'black_berry', 'black_mullberry', 'brazil_nut', 'camu_camu', 'cashew', 'cempedak', 'chenet', 'cherimoya', 'chico', 'chokeberry', 'cluster_fig', 'coconut', 'corn_kernel', 'cranberry', 'cupuacu', 'custard_apple', 'damson', 'dewberry', 'dragonfruit', 'durian', 'eggplant', 'elderberry', 'emblic', 'feijoa', 'fig', 'finger_lime', 'gooseberry', 'goumi', 'grape', 'grapefruit', 'greengage', 'grenadilla', 'guava', 'hard_kiwi', 'hawthorn', 'hog_plum', 'horned_melon', 'indian_strawberry', 'jaboticaba', 'jackfruit', 'jalapeno', 'jamaica_cherry', 'jambul', 'jocote', 'jujube', 'kaffir_lime', 'kumquat', 'lablab', 'langsat', 'longan', 'mabolo', 'malay_apple', 'mandarine', 'mango', 'mangosteen', 'medlar', 'mock_strawberry', 'morinda', 'mountain_soursop', 'oil_palm', 'olive', 'otaheit

In [3]:
#数据增强预设
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.15),
    layers.RandomZoom(0.15),
    layers.RandomContrast(0.1),
])

#导入预训练模型
base_model = EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_shape=(224,224,3)
)

#冻结基模型
base_model.trainable = False

#设计新模型
inputs = layers.Input(shape=(224,224,3))

#数据增强
x = data_augmentation(inputs)

#归一化
x = tf.keras.applications.efficientnet.preprocess_input(x)

#基模型
x = base_model(x, training=False)

#加入分类头
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(100, activation='softmax')(x)

model_3 = models.Model(inputs, outputs)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [4]:
#模型预设
model_3.compile(
    optimizer=optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

#回调设置
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-7
)


print("Training top layers only")
history_3_ENB0 = model_3.fit(
    train_dataset,
    epochs=100,
    validation_data=val_dataset,
    callbacks=[early_stopping, reduce_lr]
)

#保存模型
model_3.save('/kaggle/working/tansfer_model')

Training top layers only
Epoch 1/100


E0000 00:00:1752716576.882756      35 meta_optimizer.cc:966] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_1_1/efficientnetb0_1/block2b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1752716579.648314      99 cuda_dnn.cc:529] Loaded cuDNN version 90300


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 108s 71ms/step - accuracy: 0.0962 - loss: 4.1284 - val_accuracy: 0.5220 - val_loss: 2.0170 - learning_rate: 1.0000e-04
Epoch 2/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 70s 56ms/step - accuracy: 0.3699 - loss: 2.4573 - val_accuracy: 0.5978 - val_loss: 1.5532 - learning_rate: 1.0000e-04
Epoch 3/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 70s 56ms/step - accuracy: 0.4543 - loss: 2.0331 - val_accuracy: 0.6362 - val_loss: 1.3541 - learning_rate: 1.0000e-04
Epoch 4/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 70s 56ms/step - accuracy: 0.5038 - loss: 1.8155 - val_accuracy: 0.6564 - val_loss: 1.2433 - learning_rate: 1.0000e-04
Epoch 5/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 70s 56ms/step - accuracy: 0.5344 - loss: 1.6746 - val_accuracy: 0.6720 - val_loss: 1.1656 - learning_rate: 1.0000e-04
Epoch 6/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 69s 55ms/step - accuracy: 0.5651 - loss: 1.5502 - val_accuracy: 0.6880 - val_loss: 1.0976 - learning_rate: 1.0000e-04
Epoch 7/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 69

ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=/kaggle/working/tansfer_model.

In [6]:
#解冻微调模型
def unfreeze_model(model):
    for layer in model.layers:
        if layer.__class__.__name__ == 'BatchNormalization':
            layer.trainable = False

    # Find the EfficientNetB0 layer by class name
    base_model = None
    for layer in model.layers:
        if layer.__class__.__name__ == 'Functional': # EfficientNetB0 base model is a Functional API model
            base_model = layer
            break


    if base_model is None:
        print("Error: EfficientNetB0 layer not found in the model.")
        return

    #解冻30层基模型
    for layer in base_model.layers[-30:]:
        if layer.__class__.__name__ != 'BatchNormalization':
            layer.trainable = True


    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-5),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

print("Fine-Tuning Top Layers of Model_3")
unfreeze_model(model_3)


history_4_fine_ENB0 = model_3.fit(
    train_dataset,
    epochs=100,
    validation_data=val_dataset,
    callbacks=[early_stopping, reduce_lr]
)

#保存模型
model_3.save('/kaggle/working/transfer_model2.h5')

Fine-Tuning Top Layers of Model_3
Epoch 1/100


E0000 00:00:1752721802.609000      35 meta_optimizer.cc:966] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_1_1/efficientnetb0_1/block2b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 99s 65ms/step - accuracy: 0.8043 - loss: 0.6165 - val_accuracy: 0.7954 - val_loss: 0.7158 - learning_rate: 1.0000e-05
Epoch 2/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 77s 62ms/step - accuracy: 0.8054 - loss: 0.6203 - val_accuracy: 0.7960 - val_loss: 0.7083 - learning_rate: 1.0000e-05
Epoch 3/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 77s 62ms/step - accuracy: 0.8058 - loss: 0.6044 - val_accuracy: 0.7972 - val_loss: 0.7057 - learning_rate: 1.0000e-05
Epoch 4/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 77s 61ms/step - accuracy: 0.8137 - loss: 0.5846 - val_accuracy: 0.7964 - val_loss: 0.7071 - learning_rate: 1.0000e-05
Epoch 5/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 77s 61ms/step - accuracy: 0.8127 - loss: 0.5833 - val_accuracy: 0.7978 - val_loss: 0.7053 - learning_rate: 1.0000e-05
Epoch 6/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 77s 62ms/step - accuracy: 0.8181 - loss: 0.5730 - val_accuracy: 0.7984 - val_loss: 0.7059 - learning_rate: 1.0000e-05
Epoch 7/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 77s